In [ ]:
!pip install snowflake-ml-python

In [2]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions

In [3]:
connection_parameters = { "account": "ug94937", "user": "AAVESH", "password": "Panchi@202219", "role": "AAVESH", "warehouse": "FOSFOR_INSIGHT_WH",  "database": "FDC_DEV_AAVESH", "schema": "FDC_DEV_AAVESH","region":"us-east4.gcp"}
new_session = Session.builder.configs(connection_parameters).create()

In [4]:
df= new_session.sql("select * from HR_COMMON")
# tpot_data = df.toPandas()
from snowflake.ml.modeling.preprocessing import LabelEncoder
label_encoder = LabelEncoder(input_cols=['DEPARTMENT'],output_cols=["ENCODED_DEPARTMENT"],drop_input_cols=True)
#encoded_label1 = label_encoder.fit(tpot_data)

In [8]:
# label_encoder = label_encoder.fit(df)
# df = label_encoder.transform(df)
# df

In [11]:

#df1 = df.groupBy("DEPARTMENT").count().show()
#df1.columns

input_columns = ['SATISFACTION_LEVEL', 'LAST_EVALUATION', 'NUMBER_PROJECT',
       'AVERAGE_MONTLY_HOURS', 'TIME_SPEND_COMPANY', 'WORK_ACCIDENT', 'LEFT',
       'PROMOTION_LAST_5YEARS','ENCODED_DEPARTMENT']

from snowflake.ml.modeling.ensemble import GradientBoostingClassifier
OUTPUT_COLS = ['SALARY']
target_col= 'SALARY'
snowflake_model = GradientBoostingClassifier(input_cols=input_columns, label_cols=target_col, output_cols=OUTPUT_COLS)
from snowflake.ml.modeling.pipeline import Pipeline
pipeline = Pipeline(
    steps=[
        (
            "OE",
            label_encoder
        ),
        (
            "MMS",
            snowflake_model
        ),
    ]
)
pipeline.fit(df)
#training_features
df_remove = df.drop('SALARY')
pipeline.predict(df_remove)

/Users/10639490/Documents/Project_repo/insight-automl/snowparkml_automl/venv/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DummyClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/10639490/Documents/Project_repo/insight-automl/snowparkml_automl/venv/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/10639490/Documents/Project_repo/insight-automl/snowparkml_automl/venv

In [15]:
df_predication = pipeline.predict(df_remove)
df_predication.show()

/Users/10639490/Documents/Project_repo/insight-automl/snowparkml_automl/venv/lib/python3.9/site-packages/snowflake/ml/_internal/utils/snowpark_dataframe_utils.py:88: UserWarning: Warning: The Decimal(38, 2) data type is being automatically converted to DoubleType in the Snowpark DataFrame. This automatic conversion may lead to potential precision loss and rounding errors. If you wish to prevent this conversion, you should manually perform the necessary data type conversion.
  warnings.warn(
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ENCODED_DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"SALARY"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|7.0                   |0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |low       |
|7.0                   |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |low       |
|7.0 

In [23]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
                                                    train_size=0.75, test_size=0.25)

from sklearn.metrics import make_scorer

def my_custom_accuracy(y_true, y_pred):
    return float(sum(y_pred == y_true)) / len(y_true)

# Make a custom a scorer from the custom metric function
# Note: greater_is_better=False in make_scorer below would mean that the scoring function should be minimized.
my_custom_scorer = make_scorer(my_custom_accuracy, greater_is_better=True)

tpot_config = {
    'snowflake.ml.modeling.ensemble.gradient_boosting_classifier.GradientBoostingClassifier': {
    }
}

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2,
                      config_dict=tpot_config,scoring=my_custom_scorer)
tpot.fit(X_train, y_train)
# print(tpot.score(X_test, y_test))
# tpot.export('tpot_digits_pipeline.py')

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -inf


IndexError: list index out of range

In [19]:
str(snowflake_model)

'<snowflake.ml.modeling.ensemble.gradient_boosting_classifier.GradientBoostingClassifier object at 0x1334af940>'